In [17]:
import json
import pandas as pd
import psycopg2 as ps
from pathlib import Path

def read_json_as_df(file_path):
    """
    Reads a JSON file and converts it to a pandas DataFrame.
    
    Args:
        file_path (str): The path to the JSON file.
        
    Returns:
        pd.DataFrame: A DataFrame containing the data from the JSON file.
    """
    try:
        
        df = pd.DataFrame(file_path)
        return df
    except ValueError as e:
        print(f"Error reading JSON file: {e}")
        return None
  
creatures_json_path = Path("/home/tick/Project-Tibia/creatures.json")

with creatures_json_path.open() as f:
    creatures_data = json.load(f)

df = read_json_as_df(creatures_data['creatures']['creature_list'])

df.head()    

,name,race,image_url,featured
0,Acid Blobs,acidblob,https://static.tibia.com/images/library/acidbl...,False
1,Acolytes Of The Cult,cultacolyte,https://static.tibia.com/images/library/cultac...,False
2,Adepts Of The Cult,cultadept,https://static.tibia.com/images/library/cultad...,False
3,Adult Goannas,adultgoanna,https://static.tibia.com/images/library/adultg...,False
4,Afflicted Striders,afflictedstrider,https://static.tibia.com/images/library/afflic...,False


In [18]:
# Display the schema of the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       638 non-null    object
 1   race       638 non-null    object
 2   image_url  638 non-null    object
 3   featured   638 non-null    bool  
dtypes: bool(1), object(3)
memory usage: 15.7+ KB


In [ ]:
def connect_to_db():
    """
    Connects to the PostgreSQL database and returns the connection object.
    
    Returns:
        conn: A connection object to the PostgreSQL database.
    """
    try:
        conn = ps.connect(
            dbname="tibia_db",
            user="tibia_user",
            password="tibia_pass",
            host="172.20.0.1",
            port="5432"
        )
        
        print("Connected to the database successfully.")
        return conn
    except ps.DatabaseError as e:
        print(f"Error connecting to the database: {e}")
        return None

In [ ]:
def send_df_to_db(df, conn):
    """
    Sends a pandas DataFrame to the PostgreSQL database.
    
    Args:
        df (pd.DataFrame): The DataFrame to send.
        conn: The connection object to the PostgreSQL database.
    """
    try:
        # Create a cursor object
        cursor = conn.cursor()

        # Insert data into the 'creatures' table
        insert_query = """
        INSERT INTO creatures (name, race, image_url, featured)
        VALUES (%s, %s, %s, %s);
        """
        for _, row in df.iterrows():
            cursor.execute(insert_query, (row['name'], row['race'], row['image_url'], row['featured']))
        
        # Commit the transaction
        conn.commit()
        print("DataFrame sent to the database successfully.")
        
        # Close the cursor
        cursor.close()
    except Exception as e:
        print(f"Error sending DataFrame to the database: {e}")
        

In [27]:
send_df_to_db(df, connect_to_db())

DataFrame sent to the database successfully.
